In [83]:
from pymongo import MongoClient
import numpy as np
import pandas as pd
import pymongo
import time
import math
from pymongo import ReadPreference

In [84]:
pymongo.version

'3.3.0'

In [85]:
client4 = MongoClient(['34.196.21.20:27017', '34.197.19.183:27017', '34.193.159.184:27017']
                      , replicaset='rs0'
                      , readPreference='secondaryPreferred'
                      , localThresholdMS=35)

In [86]:
 print(client4.nodes)

frozenset({('34.197.36.27', 30000), ('34.197.19.183', 27017), ('34.196.21.20', 27017), ('34.193.159.184', 27017)})


In [87]:
print(client4.read_preference)

SecondaryPreferred(tag_sets=None)


In [88]:
client4.database_names() 

['admin', 'election', 'local']

In [89]:
db = client4['election']   
db.collection_names()

['grdelec', 'us']

In [90]:
# nom des colonnes dans la collection us: vote_id, vote_timestamp, state, vote_result

In [91]:
db.us.find_one()

{'_id': ObjectId('587d1b4e2690f54cd18630e8'),
 'state': 'Minnesota',
 'vote_id': 1,
 'vote_result': 'Clinton',
 'vote_timestamp': '2016-11-08T20:00'}

## Functions

In [92]:
# Transformation du resultat d'une requete en Dataframe
# >>> KO fonctionne qu'avec PyMongo:version 2.8.0

def query_to_dataframe(cursor):
    df = pd.DataFrame()
    if len(list(cursor)) != 0:
        for i, el in enumerate(list(cursor)):
            df = df.append(pd.DataFrame(el, index=[i]), ignore_index=True)
    return df

In [93]:
# Adaptation version Nico

def query_to_dataframe_2(cursor):
    df = pd.DataFrame()
    if len(list(cursor)) != 0:
        for i, el in enumerate(cursor):
            df = df.append(pd.DataFrame(el, index=[i]), ignore_index=True)
    return df 

In [94]:
# Requete Laura
# OK pour version 3.3.0

def query_v21(key_time):
    
    cursor=db.us.aggregate([
                { "$match": { "vote_timestamp":key_time } },
                { "$group": { "_id": { "state": "$state", "vote_result":"$vote_result"}, "total":  {"$sum":1 }}  },
              ])
    info=(list(cursor))
    ligne=[]
    df=pd.DataFrame()
    for i in range(len(info)):
        ligne=[info[i]['total'] ,info[i]['_id']['state'],info[i]['_id']['vote_result']]
        df=pd.concat([df,pd.DataFrame(ligne).T])
    df.columns=['nb_vote','state','result']

    return df

In [95]:
# Remake with Try Except

def query_v2(key_time):
      
    try:
        cursor=db.us.aggregate([
                { "$match": { "vote_timestamp":key_time } },
                { "$group": { "_id": { "state": "$state", "vote_result":"$vote_result"}, "total":  {"$sum":1 }}  },
              ])
        pass
    except pymongo.errors.AutoReconnect:
        print('Disconnecting !!!')
        cursor.skip()
        time.sleep(5)
        
    
    info=(list(cursor))
    ligne=[]
    df=pd.DataFrame()
    for i in range(len(info)):
        ligne=[info[i]['total'] ,info[i]['_id']['state'],info[i]['_id']['vote_result']]
        df=pd.concat([df,pd.DataFrame(ligne).T])
    if not df.empty:
        df.columns=['nb_vote','state','result']

    return df

In [96]:
# Requete nombre de votes par Candidat par Etat 

def query_by_time(key_time):  
    cursor = db.us.aggregate([
                 { "$match": { "vote_timestamp":key_time } },
                 { "$group": { "_id": { "state": "$state", "vote_result": "$vote_result" }
                              , "state": { "$first": '$state' }
                              , "vote_result": { "$first": '$vote_result' }
                              , "total":  {"$sum":1 }}  }, 
               ])
    return cursor

In [97]:
# Requete du nombre de Grands Electeurs par State

def query_grands_elec():
    df_grd_elec = pd.DataFrame()
    cur = db.grdelec.find()
    xcur = list(cur)
    if len(xcur):
        for j, elt in enumerate(xcur):
            df_grd_elec = df_grd_elec.append(pd.DataFrame(elt, index=[j]), ignore_index=True)
    return df_grd_elec

In [98]:
# Resultat votes par State

def states_result(key_time):
    
    #cursor = query_by_time(key_time)
    #df_state = query_to_dataframe(cursor)
    
    df_state = query_v2(key_time)
    
    df_resultat = pd.DataFrame()
    df_prop_state = pd.DataFrame()
    
    if not df_state.empty:
        df_state_group = df_state.groupby('state')
        for key, elmt in df_state_group:
            
            elmt['pct'] = elmt['nb_vote'].apply(lambda x: x/elmt.nb_vote.sum())
            
            df_resultat = df_resultat.append(elmt[elmt.pct == elmt.pct.max()][['state', 'result', 'nb_vote']], ignore_index=True)

            df_prop_state = df_prop_state.append(elmt[elmt.state == 'Maine'], ignore_index=True)
            df_prop_state = df_prop_state.append(elmt[elmt.state == 'Nebraska'], ignore_index=True)
    
    return df_resultat, df_prop_state

In [99]:
# Resultat de l'élection

def score_election(df_r, df_grd_elec):
    result = pd.merge(df_r, df_grd_elec, on='state')
    return result[['result', 'grands_electeurs']].groupby('result').sum()

In [100]:
# Resultat de l'élection par State

def score_state(df_r, df_grd_elec):
    return pd.merge(df_r, df_grd_elec, on='state')

In [101]:
# Maine et le Nebraska

def score_election_prop(df_r, df_prop_state, df_grd_elec):
    result = pd.merge(df_r, df_grd_elec, on='state')
    result[['result', 'grands_electeurs']].groupby('result').sum()

## Fonction de Test

In [ ]:
start_time = time.time()
key_time = '2016-11-08T20:00'
df_r = pd.DataFrame()
df_prop_state = pd.DataFrame()

# Boucle de simulation de l'incrémentation du temps
for el in range(60):
    
        # CRéation du timestamp format string mongodb
        key_time = '2016-11-08T20:' + ('00' + str(el))[-2:]
        print(key_time)
        print(client4.nodes)
        
        start_time_r = time.time()
        # Recupération des votes + stockage dans un Dataframe (df_r)
        df_r = df_r.append(states_result(key_time)[0], ignore_index=True)
        #print()
        print(key_time + " --- R: --- %s seconds ---" % (time.time() - start_time_r)) 
        #print()
        
        # Recupération des votes pour le calcul des états au scrutin proportionnel + stockage dans un Dataframe
        df_prop_state = df_prop_state.append(states_result(key_time)[1], ignore_index=True)
        #print(df_r)
print()
print(df_prop_state)
print("Total: --- %s seconds ---" % (time.time() - start_time)) 

2016-11-08T20:00
frozenset({('34.197.36.27', 30000), ('34.197.19.183', 27017), ('34.196.21.20', 27017), ('34.193.159.184', 27017)})


/Users/Wes/anaconda3/lib/python3.5/site-packages/ipykernel/__main__.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


2016-11-08T20:00 --- R: --- 4.616608142852783 seconds ---
2016-11-08T20:01
frozenset({('34.197.36.27', 30000), ('34.197.19.183', 27017), ('34.196.21.20', 27017), ('34.193.159.184', 27017)})
2016-11-08T20:01 --- R: --- 5.598875999450684 seconds ---
2016-11-08T20:02
frozenset({('34.197.36.27', 30000), ('34.197.19.183', 27017), ('34.196.21.20', 27017), ('34.193.159.184', 27017)})
2016-11-08T20:02 --- R: --- 13.990018129348755 seconds ---
2016-11-08T20:03
frozenset({('34.197.36.27', 30000), ('34.196.21.20', 27017), ('34.193.159.184', 27017)})
2016-11-08T20:03 --- R: --- 1.798548936843872 seconds ---
2016-11-08T20:04
frozenset({('34.197.36.27', 30000), ('34.196.21.20', 27017), ('34.193.159.184', 27017)})
2016-11-08T20:04 --- R: --- 24.07693886756897 seconds ---
2016-11-08T20:05
frozenset({('34.196.21.20', 27017), ('34.197.36.27', 30000)})
2016-11-08T20:05 --- R: --- 2.597018003463745 seconds ---
2016-11-08T20:06
frozenset({('34.196.21.20', 27017), ('34.197.36.27', 30000)})
2016-11-08T20:06 

In [21]:
df_grd_elec = query_grands_elec()
print(df_grd_elec.head())
print(df_grd_elec['grands_electeurs'].sum())

                        _id code  grands_electeurs            state
0  58823c757c12b81956bc3d8f   AZ                11          Arizona
1  58823c757c12b81956bc3d93   SC                 9  Caroline_du_Sud
2  58823c757c12b81956bc3da1   KS                 6           Kansas
3  58823c757c12b81956bc3da2   KY                 8         Kentucky
4  58823c757c12b81956bc3db0   NY                29         New_York
538


In [22]:
print(score_election(df_r, df_grd_elec))

         grands_electeurs
result                   
Clinton                10
Trump                  18
